# TOPSIS 区间版本算法教程

本教程介绍如何使用 TOPSIS（逼近理想解排序法）区间版本进行多准则决策分析。

## 目录

1. [算法简介](#1-算法简介)
2. [基础示例](#2-基础示例)
3. [区间数概念](#3-区间数概念)
4. [详细分析](#4-详细分析)
5. [敏感性分析](#5-敏感性分析)
6. [与其他算法对比](#6-与其他算法对比)

## 1. 算法简介

TOPSIS（Technique for Order Preference by Similarity to Ideal Solution）是一种基于距离的多准则决策方法。

### 核心思想

**最优方案应该：**
- 距离正理想解（PIS）最近
- 距离负理想解（NIS）最远

### 相对接近度

$$C_i = \frac{D_i^-}{D_i^+ + D_i^-}$$

其中 $D_i^+$ 是到正理想解的距离，$D_i^-$ 是到负理想解的距离。

In [ ]:
# 导入必要的库
from mcda_core.models import DecisionProblem, Criterion
from mcda_core.interval import Interval
from mcda_core.algorithms import IntervalTOPSISAlgorithm

print("导入成功!")

## 2. 基础示例

让我们从一个简单的例子开始：选择最佳云服务提供商。

In [ ]:
# 定义备选方案
alternatives = ("AWS", "Azure", "GCP")

# 定义准则
criteria = (
    Criterion(name="性能", weight=0.4, direction="higher_better"),
    Criterion(name="成本", weight=0.3, direction="lower_better"),
    Criterion(name="可靠性", weight=0.2, direction="higher_better"),
    Criterion(name="易用性", weight=0.1, direction="higher_better"),
)

# 定义区间评分
scores = {
    "AWS": {
        "性能": Interval(80.0, 90.0),
        "成本": Interval(50.0, 70.0),
        "可靠性": Interval(85.0, 95.0),
        "易用性": Interval(75.0, 85.0),
    },
    "Azure": {
        "性能": Interval(85.0, 95.0),
        "成本": Interval(40.0, 60.0),
        "可靠性": Interval(80.0, 90.0),
        "易用性": Interval(80.0, 90.0),
    },
    "GCP": {
        "性能": Interval(82.0, 92.0),
        "成本": Interval(60.0, 80.0),
        "可靠性": Interval(75.0, 85.0),
        "易用性": Interval(70.0, 80.0),
    },
}

print("备选方案:", alternatives)
print("准则数量:", len(criteria))

### 执行 TOPSIS 计算

In [ ]:
# 创建决策问题
problem = DecisionProblem(
    alternatives=alternatives,
    criteria=criteria,
    scores=scores
)

# 执行 TOPSIS 区间版本计算
algo = IntervalTOPSISAlgorithm()
result = algo.calculate(problem)

# 显示结果
print("TOPSIS 排名结果:")
print("-" * 50)
for item in result.rankings:
    print(f"第 {item.rank} 名: {item.alternative} (相对接近度: {item.score:.4f})")

## 3. 区间数概念

区间数用于表示不确定性和模糊性。

In [ ]:
# 创建区间
narrow = Interval(80.0, 85.0)   # 窄区间：高确定性
medium = Interval(70.0, 90.0)   # 中等区间
wide = Interval(50.0, 100.0)    # 宽区间：低确定性
point = Interval(75.0, 75.0)    # 点区间：确定值

print("区间数属性:")
print("-" * 50)
print(f"窄区间 {narrow}: 宽度={narrow.width:.1f}, 中点={narrow.midpoint:.1f}")
print(f"中区间 {medium}: 宽度={medium.width:.1f}, 中点={medium.midpoint:.1f}")
print(f"宽区间 {wide}: 宽度={wide.width:.1f}, 中点={wide.midpoint:.1f}")
print(f"点区间 {point}: 宽度={point.width:.1f}, 中点={point.midpoint:.1f}")

### 区间数运算

In [ ]:
# 区间运算示例
a = Interval(10.0, 20.0)
b = Interval(5.0, 15.0)

print("区间运算:")
print("-" * 50)
print(f"a = {a}")
print(f"b = {b}")
print(f"a + b = {a + b}")
print(f"a - b = {a - b}")
print(f"a * 2 = {a * 2}")
print(f"a 中点 = {a.midpoint}")

## 4. 详细分析

让我们深入分析 TOPSIS 的计算过程。

In [ ]:
# 获取元数据
metadata = result.metadata.metrics

# 理想解和负理想解
ideal = metadata["ideal"]
negative_ideal = metadata["negative_ideal"]

print("理想解和负理想解:")
print("-" * 50)
for i, crit in enumerate(criteria):
    direction_str = "效益型" if crit.direction == "higher_better" else "成本型"
    print(f"{crit.name} ({direction_str}):")
    print(f"  理想解: {ideal[i]:.4f}")
    print(f"  负理想解: {negative_ideal[i]:.4f}")

In [ ]:
# 距离分析
d_plus = metadata["distance_to_ideal"]
d_minus = metadata["distance_to_negative_ideal"]

print("距离分析:")
print("-" * 50)
print(f"{'方案':<12} {'到理想解 (D+)':<15} {'到负理想解 (D-)':<15} {'相对接近度':<12}")
print("-" * 50)

for alt in alternatives:
    closeness = d_minus[alt] / (d_plus[alt] + d_minus[alt])
    print(f"{alt:<12} {d_plus[alt]:<15.4f} {d_minus[alt]:<15.4f} {closeness:<12.4f}")

## 5. 敏感性分析

分析权重变化对排名的影响。

In [ ]:
# 定义不同的权重场景
scenarios = [
    ("原始", [0.4, 0.3, 0.2, 0.1]),
    ("性能优先", [0.6, 0.15, 0.15, 0.1]),
    ("成本优先", [0.2, 0.5, 0.2, 0.1]),
    ("平衡权重", [0.25, 0.25, 0.25, 0.25]),
]

print("权重敏感性分析:")
print("-" * 70)
print(f"{'场景':<15} {'AWS':<10} {'Azure':<10} {'GCP':<10}")
print("-" * 70)

for scenario_name, weights in scenarios:
    # 创建新准则
    new_criteria = []
    for i, crit in enumerate(criteria):
        new_criteria.append(Criterion(
            name=crit.name,
            weight=weights[i],
            direction=crit.direction
        ))
    
    # 计算新结果
    new_problem = DecisionProblem(
        alternatives=alternatives,
        criteria=tuple(new_criteria),
        scores=scores
    )
    
    new_result = algo.calculate(new_problem)
    ranking = [r.alternative for r in new_result.rankings]
    
    print(f"{scenario_name:<15} {ranking[0]:<10} {ranking[1]:<10} {ranking[2]:<10}")

## 6. 与其他算法对比

对比 TOPSIS 区间版本与精确值版本的结果。

In [ ]:
from mcda_core.algorithms import TOPSISAlgorithm

# 将区间转换为中点值
exact_scores = {}
for alt, alt_scores in scores.items():
    exact_scores[alt] = {
        crit: value.midpoint if isinstance(value, Interval) else value
        for crit, value in alt_scores.items()
    }

# 精确值版本
exact_problem = DecisionProblem(
    alternatives=alternatives,
    criteria=criteria,
    scores=exact_scores
)
exact_result = TOPSISAlgorithm().calculate(exact_problem)

# 区间版本
interval_result = algo.calculate(problem)

# 对比结果
print("版本对比:")
print("-" * 50)
print(f"{'排名':<8} {'精确值版本':<15} {'区间版本':<15}")
print("-" * 50)
for i in range(len(alternatives)):
    print(f"{i+1:<8} {exact_result.rankings[i].alternative:<15} {interval_result.rankings[i].alternative:<15}")

print("\n相对接近度对比:")
print("-" * 50)
print(f"{'方案':<12} {'精确值':<15} {'区间版本':<15}")
print("-" * 50)
for alt in alternatives:
    exact_score = next(r.score for r in exact_result.rankings if r.alternative == alt)
    interval_score = next(r.score for r in interval_result.rankings if r.alternative == alt)
    print(f"{alt:<12} {exact_score:<15.4f} {interval_score:<15.4f}")

## 总结

本教程介绍了 TOPSIS 区间版本算法的基本使用方法：

1. **区间数**：用于表示评分的不确定性
2. **核心概念**：相对接近度 = 负理想解距离 / (理想解距离 + 负理想解距离)
3. **适用场景**：需要考虑不确定性的多准则决策问题

### 选择建议

- **使用精确值版本**：当数据确定、无不确定性时
- **使用区间版本**：当数据存在不确定性或模糊性时

### 相关文档

- [TOPSIS 区间版本使用指南](./topsis_interval_guide.md)
- [TOPSIS 区间版本算法详解](./topsis_interval_deep_dive.md)